# 压强并行计算

为了对某个体系有基础认知，运行此程序以得到不同物相的压强曲线。与深入研究某个样品的run.ipynb不同，本note专注于同时测量许多样品的压强。

## 一：产生可用的原初样品。一般可用最密堆积晶格。

#### 最密堆积由于需要尝试压缩粒子，需要计算一定时间。使用下述管道。

##### 步骤1：设定粒子形状以及单胞中粒子个数，运行下面代码

In [8]:
# you may use this code for user-defined shape.
import coxeter
import rowan
import numpy as np
rhombic_dodecahedron = coxeter.families.CatalanFamily.get_shape("Rhombic Dodecahedron")

vertices = rhombic_dodecahedron.vertices

for i in range(len(vertices)):
    vertices[i] = rowan.rotate(
        rowan.from_axis_angle([0, 1, 0], np.arctan(0.56123102 / 0.39685026)),
        vertices[i],
    )

for i in range(len(vertices)):
    if vertices[i][2] < 0:
        vertices[i] = rowan.rotate(rowan.from_axis_angle([0, 0, 1], np.pi / 3), vertices[i])

In [10]:
# Densest using different pipeline
import hcpmc
import numpy as np
import os
# TODO: set the factory and the initializer
factory = hcpmc.particlefactory.Anyshape(vertices)
particle_per_cell = 2
seeds = np.arange(70)
temp = "sample0.gsd"

# init = hcpmc.initializer.Densest(factory, particle_per_cell, temp, seeds)
init = hcpmc.initializer.Expander(factory,os.getcwd()+'/sample0.gsd')

##### 步骤2：发送命令开始压缩体系，复制最后两行代码到终端中执行可以查看运行进程。

In [ ]:
init.compress()

Output: 
Error: Querying scheduler...
Submitting cluster job 'Densest/461d6be01df036c8e80cdfb1e4f3410d/compress/535a1c9518984fc54c56a2864bc0ca98':
 - Group: compress(461d6be01df036c8e80cdfb1e4f3410d)
Submitting cluster job 'Densest/10598cbdfa12d4d36b5a219cf0519529/compress/6c8bf0da73f8b85d895ae435923a4556':
 - Group: compress(10598cbdfa12d4d36b5a219cf0519529)
Submitting cluster job 'Densest/7e40e632d1d4c3838894d41c804758c3/compress/bc027b5be00a3af06d0618ffe7705752':
 - Group: compress(7e40e632d1d4c3838894d41c804758c3)
Submitting cluster job 'Densest/50c4dcf295681b73a6d2175f1b946579/compress/dd6a171cdb9a322ac7ba8a5f6ae4cb68':
 - Group: compress(50c4dcf295681b73a6d2175f1b946579)
Submitting cluster job 'Densest/7f3ce74af8e40081bf849f0dbfe7a932/compress/94645cabb47484fb7ed2b5a07db969a8':
 - Group: compress(7f3ce74af8e40081bf849f0dbfe7a932)
Submitting cluster job 'Densest/bb60d2acede9e27c9dd517cdc76917ae/compress/1205a70b8b7b0ed8eb9acec1fac0646a':
 - Group: compress(bb60d2acede9e27c9dd517cd

##### 步骤3：等待全部进程运行结束，运行下面的代码生成原初样品。

In [ ]:
init.analysis()
init.create_sample0()

## 二：生成project和job。对于流体和固体生成不同project。

### Fluid Pipeline

In [11]:
# Fluid Project
from flow import FlowProject
import os
import numpy as np
packing_fractions = np.arange(0.01, 0.5, 0.01)
replication = [8, 8, 8]
samplename = "/sample.gsd"
seeds = [12345]

class FluidPressure(FlowProject):
    pass


project = FluidPressure.init_project("FluidPressure")

for pf in packing_fractions:
    for seed in seeds:
        job = project.open_job({'pf': pf,'seed':seed}).init()
        with job:
            init.create_sample(pf, replication, job.path + samplename)
path = os.getcwd()  # get the absolute path of the current working directory
f = open(path + "/FluidPressure/project.py", "w")  # open 'project.py' file

/home/jwang/miniforge3/envs/hd5/lib/python3.13/site-packages/synced_collections/numpy_utils.py:46: NumpyConversionWarning: Any numpy types provided will be transparently converted to the closest base Python equivalents.
  warnings.warn(NUMPY_CONVERSION_WARNING, NumpyConversionWarning)


In [ ]:
# FluidPressure/project.py
# you may use this code for user-defined shape.
import coxeter
import rowan
import numpy as np

rhombic_dodecahedron = coxeter.families.CatalanFamily.get_shape("Rhombic Dodecahedron")

vertices = rhombic_dodecahedron.vertices

for i in range(len(vertices)):
    vertices[i] = rowan.rotate(
        rowan.from_axis_angle([0, 1, 0], np.arctan(0.56123102 / 0.39685026)),
        vertices[i],
    )

for i in range(len(vertices)):
    if vertices[i][2] < 0:
        vertices[i] = rowan.rotate(
            rowan.from_axis_angle([0, 0, 1], np.pi / 3), vertices[i]
        )
# Check this and DON'T RUN this block.
from flow import FlowProject
import os
import sys
path = os.getcwd() + "/.."
sys.path.append(os.getcwd() + "/..")
import hcpmc

# TODO: Determine the sample, factory, steps and seed.
# TODO: Then you copy this block to a file named 'project.py' in 'Pressure' directory.
samplename = "/sample.gsd"
factory = hcpmc.particlefactory.Anyshape(vertices)
equilibrium_steps = int(1e5)
sampling_steps = int(1e5)


# NO NEED TO CHANGE THIS
class FluidPressure(FlowProject):
    pass


@FluidPressure.label
def finish(job):
    return "Pressure" in job.document


@FluidPressure.post(finish)
@FluidPressure.operation(directives={"memory": "1g", "walltime": 24})
def sample(job):
    with job:
        P = hcpmc.equilibrium.fluid.Pressure(job.path + samplename, factory)
        P.calculate(equilibrium_steps, sampling_steps, job.sp.seed)
        job.doc["Pressure"] = P.get_pressure()


if __name__ == "__main__":
    FluidPressure().main()

In [15]:
# NO NEED TO CHANGE THIS
%cd FluidPressure
!rm slurm*
!python project.py submit -o sample
%cd ..
path = os.getcwd() + '/FluidPressure'
print('cd '+path)
print('watch -n 1 python project.py status')


/home/jwang/HOOMD_HPMC/amazing_codetest/FluidPressure
Querying scheduler...
Submitting cluster job 'FluidPressur/67c10c161d1f4047b6d908cc5bba41ec/sample/0929857e71e7d2363bec606a7eba1b89':
 - Group: sample(67c10c161d1f4047b6d908cc5bba41ec)
Submitting cluster job 'FluidPressur/7eb4e24a6528e7ca18a5b0853d038ccc/sample/0b5389f01b101e08dd2670a0b1d7da6f':
 - Group: sample(7eb4e24a6528e7ca18a5b0853d038ccc)
Submitting cluster job 'FluidPressur/6416438afada8638a8510e0e45c0a317/sample/15a6ce72956aef100452a8d53ec5613e':
 - Group: sample(6416438afada8638a8510e0e45c0a317)
Submitting cluster job 'FluidPressur/8f86f09eb42dafb893039e88da9e4864/sample/4fe6ed55511c408f192e26955dbfee0d':
 - Group: sample(8f86f09eb42dafb893039e88da9e4864)
Submitting cluster job 'FluidPressur/070476be29741e42d895736ff43de22c/sample/0938710c6abbef967ad5d784e108ddcc':
 - Group: sample(070476be29741e42d895736ff43de22c)
Submitting cluster job 'FluidPressur/3a7d12e98b15545a634e0d56ec71345b/sample/1af84fc05518bd9ca6b2910a0ccab9e5

### Solid Pipeline

In [17]:
# Solid Project
from flow import FlowProject
import os
import numpy as np
packing_fractions = np.arange(0.4, 0.95, 0.01)
replication = [8, 8, 8]
samplename = "/sample.gsd"
seeds = [12345]

class SolidPressure(FlowProject):
    pass


project = SolidPressure.init_project("SolidPressure")

for pf in packing_fractions:
    for seed in seeds:
        job = project.open_job({'pf': pf,'seed':seed}).init()
        with job:
            init.create_sample(pf, replication, job.path + samplename)
path = os.getcwd()  # get the absolute path of the current working directory
f = open(path + "/SolidPressure/project.py", "w")  # open 'project.py' file

/home/jwang/miniforge3/envs/hd5/lib/python3.13/site-packages/synced_collections/numpy_utils.py:46: NumpyConversionWarning: Any numpy types provided will be transparently converted to the closest base Python equivalents.
  warnings.warn(NUMPY_CONVERSION_WARNING, NumpyConversionWarning)


In [ ]:
# SolidPressure/project.py
# you may use this code for user-defined shape.
import coxeter
import rowan
import numpy as np

rhombic_dodecahedron = coxeter.families.CatalanFamily.get_shape("Rhombic Dodecahedron")

vertices = rhombic_dodecahedron.vertices

for i in range(len(vertices)):
    vertices[i] = rowan.rotate(
        rowan.from_axis_angle([0, 1, 0], np.arctan(0.56123102 / 0.39685026)),
        vertices[i],
    )

for i in range(len(vertices)):
    if vertices[i][2] < 0:
        vertices[i] = rowan.rotate(
            rowan.from_axis_angle([0, 0, 1], np.pi / 3), vertices[i]
        )
# Check this and DON'T RUN this block.
from flow import FlowProject
import os
import sys
path = os.getcwd() + "/.."
sys.path.append(os.getcwd() + "/..")
import hcpmc

# TODO: Determine the sample, factory, steps and seed.
# TODO: Then you copy this block to a file named 'project.py' in 'Pressure' directory.
samplename = "/sample.gsd"
factory = hcpmc.particlefactory.Anyshape(vertices)
equilibrium_steps = int(1e6)
sampling_steps = int(1e5)


# NO NEED TO CHANGE THIS
class SolidPressure(FlowProject):
    pass


@SolidPressure.label
def finish(job):
    return "Pressure" in job.document


@SolidPressure.post(finish)
@SolidPressure.operation(directives={"memory": "1g", "walltime": 24})
def sample(job):
    with job:
        P = hcpmc.equilibrium.solid.Pressure(job.path + samplename, factory)
        P.calculate(equilibrium_steps, sampling_steps, job.sp.seed)
        job.doc["Pressure"] = P.get_pressure()


if __name__ == "__main__":
    SolidPressure().main()

In [18]:
# NO NEED TO CHANGE THIS
%cd SolidPressure
!rm slurm*
!python project.py submit -o sample
%cd ..
path = os.getcwd() + '/SolidPressure'
print('cd '+path)
print('watch -n 1 python project.py status')


/home/jwang/HOOMD_HPMC/amazing_codetest/SolidPressure
rm: cannot remove 'slurm*': No such file or directory
Querying scheduler...
Submitting cluster job 'SolidPressur/da7e0ba035579768d3e2b5626bb459ec/sample/836069dde14aaaa197c5a8f5835ac12c':
 - Group: sample(da7e0ba035579768d3e2b5626bb459ec)
Submitting cluster job 'SolidPressur/a2a4fe8eb8b227003cba52466bc697e1/sample/e297b31031745bce695878cf236f86f2':
 - Group: sample(a2a4fe8eb8b227003cba52466bc697e1)
Submitting cluster job 'SolidPressur/0ec73f3e671b3ff985587ca6d3b7ff76/sample/1524781c47aef1643c3c29055c88d36a':
 - Group: sample(0ec73f3e671b3ff985587ca6d3b7ff76)
Submitting cluster job 'SolidPressur/8a9330f506b717e57eca118d4675413d/sample/74bfbec803042879b422e4708184ff90':
 - Group: sample(8a9330f506b717e57eca118d4675413d)
Submitting cluster job 'SolidPressur/cb8c5a2c8ac0fe3d4247d5954faebdfa/sample/c8b02936b0d937f3d21451ccf15f023b':
 - Group: sample(cb8c5a2c8ac0fe3d4247d5954faebdfa)
Submitting cluster job 'SolidPressur/34848daf3cea1f3256

## 三、分析数据

In [ ]:
import signac
import matplotlib.pyplot as plt
import numpy as np

project = signac.get_project("FluidPressure")
fluid = dict()
fluid_err = dict()
for job in project:
    fluid[job.sp.pf] = job.doc["Pressure"].split()[0]
    fluid_err[job.sp.pf] = job.doc["Pressure"].split()[2]
pfs = np.array(sorted(fluid.keys()))
fluid = np.array([fluid[pf] for pf in pfs])
fluid_err = np.array([fluid_err[pf] for pf in pfs])
plt.errorbar(pfs,fluid,yerr=fluid_err,label="Fluid")

project = signac.get_project("SolidPressure")
solid = dict()
solid_err = dict()
for job in project:
    solid[job.sp.pf] = job.doc["Pressure"].split()[0]
    solid_err[job.sp.pf] = job.doc["Pressure"].split()[2]
pfs = np.array(sorted(solid.keys()))
solid = np.array([solid[pf] for pf in pfs])
solid_err = np.array([solid_err[pf] for pf in pfs])
plt.errorbar(pfs,solid,yerr=solid_err,label="Solid")

plt.xlabel("Packing fraction")
plt.ylabel("Pressure")
plt.legend()